# 🎙️ **Create your first ARTAgent (Real-Time Voice Agent)**

**What You'll Learn**

1. **ARTAgent Framework Architecture** - Deep dive into the voice-to-voice pipeline
2. **Single Agent vs Multi-Agent Patterns** - When to use each approach
3. **Agent Configuration** - YAML-based setup and best practices
4. **Creating Your First Agent** - Hands-on Customer Support Agent implementation
5. **Tools & Prompts** - Extending agent capabilities with custom functions

**By the end of this notebook we should understand**:

- ✅ ARTAgent framework components and data flow
- ✅ Build a fully functional Customer Support Agent
- ✅ Create custom tools for business logic
- ✅ Deploy and test your agent in real conversations
- ✅ Know how to scale to multi-agent architectures

In [1]:
# 📂 Setup Working Directory for ARTAgent Framework Access
import logging
import os

# Configure logging to track directory changes
logging.basicConfig(level=logging.INFO)

# Navigate to the project root directory
# This ensures we can import ARTAgent framework modules properly
try:
    # Move up two directories from samples/hello_world/ to project root
    os.chdir("../../")
    
    # Allow override via environment variable for different setups
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    
    # Verify the target directory exists before changing
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        print(f"✅ Changed directory to: {os.getcwd()}")
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        print(f"❌ Directory does not exist: {target_directory}")
        logging.error(f"Directory does not exist: {target_directory}")
        
except Exception as e:
    print(f"❌ Error changing directory: {e}")
    logging.exception(f"An error occurred while changing directory: {e}")

# Verify we're in the correct location
print(f"📁 Current working directory: {os.getcwd()}")
print(f"📋 Contents: {', '.join(os.listdir('.')[:10])}...")

INFO:root:Successfully changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent


✅ Changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent
📁 Current working directory: c:\Users\pablosal\Desktop\gbb-ai-audio-agent
📋 Contents: .azure, .devcontainer, .env, .env.sample, .files, .git, .github, .gitignore, .pre-commit-config.yaml, .pytest_cache...


## **Prerequisites**

Before we start building our first ARTAgent, make sure you have the following setup:

**🔧 Environment Setup**

1. **Python 3.11+** - Required for the ARTAgent framework
2. **Dependencies** - Install the required packages:

```bash
pip install -r requirements.txt
```

**📂 Project Structure**

The code below automatically sets up the correct working directory for the notebook to access the ARTAgent framework and all dependencies.

### **1. ARTAgent Framework Architecture**


The full ARTAgent system connects callers through Azure Communication Services:

```
📞 Caller ↔ 🌐 ACS ↔ 🔊 STT ↔ 🤖 ARTAgent ↔ 🔊 TTS ↔ 🌐 ACS ↔ 📞 Caller
```

**Full System Components:**

| Component | Purpose | Technology |
|-----------|---------|------------|
| **📞 Caller** | Phone/Web user | Phone system, WebRTC browser |
| **🌐 ACS** | Audio streaming & call management | Azure Communication Services |
| **🧠 Orchestrator** | ARTAgent manager | Custom Framework (e.g., Semantic Kernel) |
| **🎤 STT Pipeline** | Speech-to-text conversion | Azure Speech Services |
| **🤖 ARTAgent** | Business logic & conversation | LLM/SLM + Tools + Memory |
| **🔊 TTS Pipeline** | Text-to-speech synthesis | Azure Speech Services |

#### **ARTAgent - The Single Agent Architecture:**

ARTAgent is the core compute unit - a single agent that specializes in a specific domain or task. Instead of routing between multiple agents, ARTAgent focuses on one job exceptionally well. Within the ARTAgent component, here's how the pieces work together:

```
🤖 ARTAgent
├── 💾 Memory Manager (conversation history)
├── 📝 Prompt Manager (template rendering)  
└── 🛠️ Tool System (business functions)
```

#### **How ARTAgent Integrates with the System:**

1. **🤖 ARTAgent receives from STT:**
- Transcribed user speech as text
- Audio metadata and confidence scores
- Language detection information

2. **🤖 ARTAgent processes internally:**
- **Memory Manager** → Loads conversation history
- **Prompt Manager** → Renders template with context
- **LLM** → Processes user input, decides on actions
- **Tool System** → Executes business functions if needed
- **LLM** → Formats final response text

3. **🤖 ARTAgent sends to TTS:**
- Formatted response text
- Voice settings and emotion markers
- Speech rate and style preferences

4. **Single Agent Pattern in Full System:**

```
📞 Caller → ACS → STT → ARTAgent → TTS → ACS → 📞 Caller
                        ↕️
                  [Memory + Tools]
```

In more detail...understanding how ARTAgent the conversation within the full caller-to-caller pipeline:

1. **📞 Caller speaks** → Voice travels through phone/web
2. **🌐 ACS receives** → Audio streaming to Azure infrastructure  
3. **🎤 STT processes** → Converts speech to text
4. **🤖 ARTAgent runs:**
   - ** Memory Manager** → Retrieves conversation history
   - **📝 Prompt Manager** → Builds context with templates
   - **🧠 LLM** → Processes user intent, decides actions
   - **🛠️ Tool System** → Executes business functions (if needed)
   - **🧠 LLM** → Generates response text
5. **🔊 TTS synthesizes** → Converts text back to speech
6. **🌐 ACS delivers** → Audio streaming back to caller
7. **📞 Caller hears** → Natural conversation continues

**Key Benefits of this Pattern:**
- 🎯 **End-to-end control** from caller to response
- ⚡ **Low level proccessing** without framework agent routing overhead
- 🔧 **Focused expertise** for specific domains
- 🔄 **Seamless integration** with Azure services

## 🚀 **1. Creating Our first Single ARTAgent**

Let's create a **Customer Support Agent** using the single agent pattern - one focused agent with specific tools and expertise.

In [1]:
import sys
import os
from pathlib import Path
import yaml
import asyncio
from datetime import datetime

# Add the backend to Python path
project_root = Path.cwd()  
agent_folder = project_root / 'samples' / 'hello_world'
sys.path.insert(0, str(agent_folder))

print(f"📂 Agent root: {agent_folder}")

📂 Agent root: c:\Users\pablosal\Desktop\gbb-ai-audio-agent\samples\hello_world\samples\hello_world


In [6]:
# Step 1: Create agent configuration (YAML)
customer_support_config = {
    "agent": {
        "name": "CustomerSupportAgent",
        "creator": "Tutorial User",
        "organization": "Demo Corp",
        "description": (
            "Handles customer support inquiries including product questions, "
            "order status, returns, and basic troubleshooting. Escalates "
            "complex issues to human agents when needed."
        )
    },
    "model": {
        "deployment_id": "gpt-4o-mini",
        "temperature": 0.7,
        "top_p": 0.9,
        "max_tokens": 2048
    },
    "voice": {
        "name": "en-US-Ava:DragonHDLatestNeural",
        "style": "chat",
        "rate": "+3%"
    },
    "prompts": {
        "path": "customer_support_agent.jinja"
    },
    "tools": [
        "search_product_catalog",
        "check_order_status", 
        "create_return_request",
        "escalate_to_human"
    ]
}

# Create directories and save configuration to YAML file
config_dir = project_root / "samples" / "hello_world" / "agents" / "agent_store"
config_file = config_dir / "customer_support_agent.yaml"

# Create directory if it doesn't exist
os.makedirs(config_dir, exist_ok=True)

# Write YAML configuration file
with open(config_file, 'w') as f:
    yaml.dump(customer_support_config, f, default_flow_style=False, indent=2)

print(f"💾 YAML config saved to: {config_file}")
print(f"✅ Config file exists: {config_file.exists()}")
print("\n📋 Agent Configuration:")
print(yaml.dump(customer_support_config, default_flow_style=False, indent=2))

💾 YAML config saved to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent\samples\hello_world\agents\agent_store\customer_support_agent.yaml
✅ Config file exists: True

📋 Agent Configuration:
agent:
  creator: Tutorial User
  description: Handles customer support inquiries including product questions, order
    status, returns, and basic troubleshooting. Escalates complex issues to human
    agents when needed.
  name: CustomerSupportAgent
  organization: Demo Corp
model:
  deployment_id: gpt-4o-mini
  max_tokens: 2048
  temperature: 0.7
  top_p: 0.9
prompts:
  path: customer_support_agent.jinja
tools:
- search_product_catalog
- check_order_status
- create_return_request
- escalate_to_human
voice:
  name: en-US-Ava:DragonHDLatestNeural
  rate: +3%
  style: chat



In [ ]:
# Step 1.1: Verify and display the YAML file content
print("📋 YAML Configuration Content:")
print("=" * 50)

# Read and display the actual saved YAML file
try:
    with open(config_file, 'r') as f:
        yaml_content = f.read()
    print(yaml_content)
    print("=" * 50)
    print(f"✅ YAML file successfully written with {len(yaml_content)} characters")
    
    # Also verify YAML can be parsed
    with open(config_file, 'r') as f:
        parsed_yaml = yaml.safe_load(f)
    print(f"✅ YAML file successfully parsed with {len(parsed_yaml)} sections")
    
except Exception as e:
    print(f"❌ Error reading YAML file: {e}")
    
# Show the file system location
print(f"\n📁 File location: {config_file}")
print(f"📊 File size: {config_file.stat().st_size} bytes")
print(f"📅 File modified: {config_file.stat().st_mtime}")

In [5]:
# Step 2: Create the prompt template
customer_support_prompt = """
You are a helpful customer support agent for Demo Corp. Your role is to:

🎯 **Primary Responsibilities:**
- Answer product questions accurately and helpfully
- Help customers check order status and tracking
- Process return and exchange requests
- Provide basic troubleshooting guidance
- Escalate complex issues to human agents when needed

🗨️ **Communication Style:**
- Be friendly, professional, and empathetic
- Use clear, concise language
- Always confirm understanding before taking action
- Provide specific next steps when possible

🛠️ **Available Tools:**
- `search_product_catalog`: Find product information, specs, pricing
- `check_order_status`: Look up order details and shipping status  
- `create_return_request`: Initiate return/exchange process
- `escalate_to_human`: Transfer to live agent for complex issues

🚫 **Important Constraints:**
- Only use the tools provided - do not make up information
- If you cannot help with a request, escalate to a human agent
- Always verify customer identity before accessing order information
- Be honest about limitations and processing times

{% if conversation_history %}
📋 **Conversation History:**
{{ conversation_history }}
{% endif %}

Customer: {{ user_message }}

How can I help you today?
"""

# Create directories and save prompt template
prompt_dir = project_root / "samples" / "hello_world" / "agents" / "prompt_store" / "templates"
prompt_file = prompt_dir / "customer_support_agent.jinja"

# Create directory if it doesn't exist
os.makedirs(prompt_dir, exist_ok=True)

# Write prompt template file
with open(prompt_file, 'w') as f:
    f.write(customer_support_prompt)

print(f"📝 Prompt template saved to: {prompt_file}")
print(f"✅ Prompt file exists: {prompt_file.exists()}")
print(f"📄 File size: {prompt_file.stat().st_size} bytes")
print("\n📋 Prompt Template Preview:")
print(customer_support_prompt[:300] + "...")

📝 Prompt template saved to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent\samples\hello_world\agents\prompt_store\templates\customer_support_agent.jinja
✅ Prompt file exists: True
📄 File size: 1310 bytes

📋 Prompt Template Preview:

You are a helpful customer support agent for Demo Corp. Your role is to:

🎯 **Primary Responsibilities:**
- Answer product questions accurately and helpfully
- Help customers check order status and tracking
- Process return and exchange requests
- Provide basic troubleshooting guidance
- Escalate c...


In [ ]:
# Step 2.1: Verify and display the Jinja template file content
print("📝 Jinja Template Content:")
print("=" * 50)

# Read and display the actual saved template file
try:
    with open(prompt_file, 'r') as f:
        template_content = f.read()
    print(template_content)
    print("=" * 50)
    print(f"✅ Template file successfully written with {len(template_content)} characters")
    
    # Verify Jinja template syntax
    from jinja2 import Template
    template_obj = Template(template_content)
    print(f"✅ Jinja template syntax is valid")
    
    # Test template rendering with sample data
    sample_render = template_obj.render(
        conversation_history="Sample conversation...",
        user_message="Hello, I need help with my order"
    )
    print(f"✅ Template renders successfully (output length: {len(sample_render)} chars)")
    
except Exception as e:
    print(f"❌ Error reading template file: {e}")
    
# Show the file system location
print(f"\n📁 File location: {prompt_file}")
print(f"📊 File size: {prompt_file.stat().st_size} bytes")
print(f"📅 File modified: {prompt_file.stat().st_mtime}")

In [6]:
# Step 4: Define agent tools

import json
from typing import Dict, Any, List

# Mock database for demonstration
PRODUCT_CATALOG = {
    "laptop-001": {
        "name": "Demo Laptop Pro",
        "price": 1299.99,
        "specs": "Intel i7, 16GB RAM, 512GB SSD",
        "availability": "In Stock"
    },
    "phone-001": {
        "name": "Demo Phone X", 
        "price": 899.99,
        "specs": "6.1 inch display, 128GB storage",
        "availability": "Limited Stock"
    }
}

ORDER_DATABASE = {
    "ORD123456": {
        "status": "Shipped",
        "tracking": "1Z999AA1234567890",
        "items": ["Demo Laptop Pro"],
        "estimated_delivery": "2024-08-22"
    },
    "ORD789012": {
        "status": "Processing",
        "tracking": None,
        "items": ["Demo Phone X"],
        "estimated_delivery": "2024-08-25"
    }
}

async def search_product_catalog(params: Dict[str, Any]) -> str:
    """
    Search the product catalog for information.
    
    Args:
        params: {"query": "search term or product ID"}
    
    Returns:
        JSON string with product information
    """
    query = params.get("query", "").lower()
    
    # Search by product ID
    if query in PRODUCT_CATALOG:
        product = PRODUCT_CATALOG[query]
        return json.dumps({
            "success": True,
            "product": product,
            "message": f"Found product: {product['name']}"
        })
    
    # Search by name
    matches = []
    for pid, product in PRODUCT_CATALOG.items():
        if query in product["name"].lower():
            matches.append({"id": pid, **product})
    
    if matches:
        return json.dumps({
            "success": True,
            "products": matches,
            "message": f"Found {len(matches)} matching products"
        })
    
    return json.dumps({
        "success": False,
        "message": "No products found matching your search"
    })

async def check_order_status(params: Dict[str, Any]) -> str:
    """
    Check the status of a customer order.
    
    Args:
        params: {"order_id": "ORD123456"}
    
    Returns:
        JSON string with order status information
    """
    order_id = params.get("order_id", "").upper()
    
    if order_id in ORDER_DATABASE:
        order = ORDER_DATABASE[order_id]
        return json.dumps({
            "success": True,
            "order": order,
            "message": f"Order {order_id} status: {order['status']}"
        })
    
    return json.dumps({
        "success": False,
        "message": "Order not found. Please check your order number."
    })

async def create_return_request(params: Dict[str, Any]) -> str:
    """
    Create a return request for a customer.
    
    Args:
        params: {"order_id": "ORD123456", "reason": "defective item"}
    
    Returns:
        JSON string with return request confirmation
    """
    order_id = params.get("order_id", "")
    reason = params.get("reason", "No reason provided")
    
    # Generate return request ID
    return_id = f"RET{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    return json.dumps({
        "success": True,
        "return_id": return_id,
        "order_id": order_id,
        "reason": reason,
        "message": f"Return request {return_id} created successfully. You will receive return instructions via email."
    })

async def escalate_to_human(params: Dict[str, Any]) -> str:
    """
    Escalate the conversation to a human agent.
    
    Args:
        params: {"reason": "complex technical issue"}
    
    Returns:
        JSON string with escalation confirmation
    """
    reason = params.get("reason", "Customer requested human assistance")
    
    return json.dumps({
        "success": True,
        "escalated": True,
        "reason": reason,
        "message": "I'm connecting you with a human agent who can better assist you. Please hold for a moment."
    })

print("🛠️ Agent tools created successfully!")
print("\n📋 Available Tools:")
print("- search_product_catalog: Find product information")
print("- check_order_status: Look up order details")
print("- create_return_request: Process returns")
print("- escalate_to_human: Transfer to live agent")

🛠️ Agent tools created successfully!

📋 Available Tools:
- search_product_catalog: Find product information
- check_order_status: Look up order details
- create_return_request: Process returns
- escalate_to_human: Transfer to live agent


In [7]:
# Test our tools with sample data
print("🧪 Testing Agent Tools:\n")

# Test product search
result1 = await search_product_catalog({"query": "laptop"})
print("📱 Product Search Result:")
print(json.dumps(json.loads(result1), indent=2))

print("\n" + "="*50 + "\n")

# Test order status
result2 = await check_order_status({"order_id": "ORD123456"})
print("📦 Order Status Result:")
print(json.dumps(json.loads(result2), indent=2))

print("\n" + "="*50 + "\n")

# Test return request
result3 = await create_return_request({"order_id": "ORD123456", "reason": "Wrong size"})
print("🔄 Return Request Result:")
print(json.dumps(json.loads(result3), indent=2))

🧪 Testing Agent Tools:

📱 Product Search Result:
{
  "success": true,
  "products": [
    {
      "id": "laptop-001",
      "name": "Demo Laptop Pro",
      "price": 1299.99,
      "specs": "Intel i7, 16GB RAM, 512GB SSD",
      "availability": "In Stock"
    }
  ],
  "message": "Found 1 matching products"
}


📦 Order Status Result:
{
  "success": true,
  "order": {
    "status": "Shipped",
    "tracking": "1Z999AA1234567890",
    "items": [
      "Demo Laptop Pro"
    ],
    "estimated_delivery": "2024-08-22"
  },
  "message": "Order ORD123456 status: Shipped"
}


🔄 Return Request Result:
{
  "success": true,
  "return_id": "RET20250820000535",
  "order_id": "ORD123456",
  "reason": "Wrong size",
  "message": "Return request RET20250820000535 created successfully. You will receive return instructions via email."
}


In [9]:
# Step 5: Create Tool Registry and Schema Functions

from typing import Any, Callable, Dict, List

# Define tool schemas for OpenAI function calling
search_product_catalog_schema = {
    "name": "search_product_catalog",
    "description": "Search the product catalog for information about products including specs, pricing, and availability",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search term or product ID to look up in catalog"
            }
        },
        "required": ["query"]
    }
}

check_order_status_schema = {
    "name": "check_order_status", 
    "description": "Check the status and tracking information for a customer order",
    "parameters": {
        "type": "object",
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The order ID to look up (e.g., ORD123456)"
            }
        },
        "required": ["order_id"]
    }
}

create_return_request_schema = {
    "name": "create_return_request",
    "description": "Create a return request for a customer order",
    "parameters": {
        "type": "object", 
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The order ID for the return request"
            },
            "reason": {
                "type": "string", 
                "description": "Reason for the return (e.g., defective item, wrong size)"
            }
        },
        "required": ["order_id", "reason"]
    }
}

escalate_to_human_schema = {
    "name": "escalate_to_human",
    "description": "Escalate the conversation to a human agent when the issue is too complex",
    "parameters": {
        "type": "object",
        "properties": {
            "reason": {
                "type": "string",
                "description": "Reason for escalation (e.g., complex technical issue, customer request)"
            }
        },
        "required": ["reason"]
    }
}

# Function mapping registry
function_mapping: Dict[str, Callable[..., Any]] = {
    "search_product_catalog": search_product_catalog,
    "check_order_status": check_order_status, 
    "create_return_request": create_return_request,
    "escalate_to_human": escalate_to_human,
}

# Available tools for OpenAI function calling
available_tools: List[Dict[str, Any]] = [
    {"type": "function", "function": search_product_catalog_schema},
    {"type": "function", "function": check_order_status_schema},
    {"type": "function", "function": create_return_request_schema},
    {"type": "function", "function": escalate_to_human_schema},
]

# Tool registry for easy lookup
TOOL_REGISTRY: Dict[str, Dict] = {
    tool["function"]["name"]: tool for tool in available_tools
}

print("🔧 Tool Registry Created Successfully!")
print(f"\n📋 Registered Tools ({len(TOOL_REGISTRY)}):")
for tool_name in TOOL_REGISTRY.keys():
    print(f"  - {tool_name}")

🔧 Tool Registry Created Successfully!

📋 Registered Tools (4):
  - search_product_catalog
  - check_order_status
  - create_return_request
  - escalate_to_human


### 🤖 **It's time...Let's Initialize our first ARTAgent**

In [ ]:
# Step 6: Initialize the RT Agent
try:
    # Import ARTAgent class
    from samples.hello_world.agents.base import ARTAgent
    # look at the import in the appa -> so it is identical
    # from apps.rtagent.backend.src.agents.artagent.base import ARTAgent
    
    print("✅ Successfully imported ARTAgent framework")
    
    # Use the actual saved file paths
    config_path = config_file  # YAML config file we just created
    template_dir = prompt_dir.parent  # templates directory
    
    print(f"🗂️ Using config file: {config_path}")
    print(f"📁 Using template directory: {template_dir}")
    print(f"✅ Config exists: {config_path.exists()}")
    print(f"✅ Template dir exists: {template_dir.exists()}")

    # Initialize the agent
    customer_agent = ARTAgent(
        config_path=str(config_path),
        template_dir=str(template_dir)
    )
    
    print("\n🤖 Agent Initialized Successfully!")
    print(f"📛 Agent Name: {customer_agent.name}")
    print(f"🏢 Organization: {customer_agent.organization}")
    print(f"📝 Description: {customer_agent.description}")
    print(f"🧠 Model: {customer_agent.model_id}")
    print(f"🎤 Voice: {customer_agent.voice_name}")
    print(f"🛠️ Tools: {len(customer_agent.tools)} tools available")
    
except ImportError as e:
    print(f"❌ Import Error: {e}")
    print("💡 Make sure you're running this from the correct environment with all dependencies installed.")
except Exception as e:
    print(f"❌ Error: {e}")
    print(f"💡 Error type: {type(e).__name__}")
    print(f"💡 Config path: {config_path if 'config_path' in locals() else 'not set'}")
    print(f"💡 Template dir: {template_dir if 'template_dir' in locals() else 'not set'}")

[2025-08-20 01:04:03,450] INFO - rt_agent: ✅ Using LOCAL tool registry from samples/hello_world/agents/tool_store
 INFO - rt_agent: ✅ Using LOCAL tool registry from samples/hello_world/agents/tool_store


INFO:rt_agent:✅ Using LOCAL tool registry from samples/hello_world/agents/tool_store


✅ Successfully imported ARTAgent framework
🗂️ Using config file: c:\Users\pablosal\Desktop\gbb-ai-audio-agent\samples\hello_world\agents\agent_store\customer_support_agent.yaml
📁 Using template directory: c:\Users\pablosal\Desktop\gbb-ai-audio-agent\samples\hello_world\agents\prompt_store
✅ Config exists: True
✅ Template dir exists: True

🗂️ Using config file: c:\Users\pablosal\Desktop\gbb-ai-audio-agent\samples\hello_world\agents\agent_store\customer_support_agent.yaml
📁 Using template directory: c:\Users\pablosal\Desktop\gbb-ai-audio-agent\samples\hello_world\agents\prompt_store
✅ Config exists: True
✅ Template dir exists: True


[2025-08-20 01:04:03,533] INFO - rt_agent INFO - rt_agent: 🛠️ Loaded 4 local tools: ['search_product_catalog', 'check_order_status', 'create_return_request', 'escalate_to_human']
INFO:rt_agent:🛠️ Loaded 4 local tools: ['search_product_catalog', 'check_order_status', 'create_return_request', 'escalate_to_human']
: 🛠️ Loaded 4 local tools: ['search_product_catalog', 'check_order_status', 'create_return_request', 'escalate_to_human']
INFO:rt_agent:🛠️ Loaded 4 local tools: ['search_product_catalog', 'check_order_status', 'create_return_request', 'escalate_to_human']


Templates found: ['templates/customer_support_agent.jinja']



[2025-08-20 01:04:03,568] INFO - rt_agent: Loaded agent 'CustomerSupportAgent' | org='Demo Corp' | desc='Handles customer support inquiries including product…' | model=gpt-4o-mini | voice=en-US-Ava:DragonHDLatestNeural/chat@+3% | prompt=customer_support_agent.jinja | tools=['search_product_catalog', 'check_order_status', 'create_return_request', 'escalate_to_human']
 INFO - rt_agent: Loaded agent 'CustomerSupportAgent' | org='Demo Corp' | desc='Handles customer support inquiries including product…' | model=gpt-4o-mini | voice=en-US-Ava:DragonHDLatestNeural/chat@+3% | prompt=customer_support_agent.jinja | tools=['search_product_catalog', 'check_order_status', 'create_return_request', 'escalate_to_human']
INFO:rt_agent:Loaded agent 'CustomerSupportAgent' | org='Demo Corp' | desc='Handles customer support inquiries including product…' | model=gpt-4o-mini | voice=en-US-Ava:DragonHDLatestNeural/chat@+3% | prompt=customer_support_agent.jinja | tools=['search_product_catalog', 'check_order_st


🤖 Agent Initialized Successfully!
📛 Agent Name: CustomerSupportAgent
🏢 Organization: Demo Corp
📛 Agent Name: CustomerSupportAgent
🏢 Organization: Demo Corp
📝 Description: Handles customer support inquiries including product questions, order status, returns, and basic troubleshooting. Escalates complex issues to human agents when needed.
🧠 Model: gpt-4o-mini
🎤 Voice: en-US-Ava:DragonHDLatestNeural
📝 Description: Handles customer support inquiries including product questions, order status, returns, and basic troubleshooting. Escalates complex issues to human agents when needed.
🧠 Model: gpt-4o-mini
🎤 Voice: en-US-Ava:DragonHDLatestNeural
🛠️ Tools: 4 tools available

🛠️ Tools: 4 tools available
